In [1]:
#import functions as func
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib as mpl
import matplotlib.colors as colors
import time
import math
from matplotlib.widgets import Slider, Button, RadioButtons
#from SlideView import SlideView
import seaborn as sns
import time
import locale
import analysis
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
# Loading weather state data (histograms)
pctaulist = []
tmphist = []
count = 0
with open('../code/pc_tau_oldData.txt') as pctauf:
    for line in pctauf:
        for value in line.split():
            tmphist.append(float(value))
            count += 1
        if count == 42:
            count = 0 
            pctaulist.append(np.asarray(tmphist).reshape(7, 6))
            tmphist = []

pctau_histograms = np.asarray(pctaulist)

# Transform pctau_histograms to tau_pc to match pctaudist (matrix transpose)
taupc_histograms = np.copy(pctau_histograms).reshape((12, 6, 7))
for i in range(pctau_histograms.shape[0]):
    taupc_histograms[i,:,:] = pctau_histograms[i,:,:].T
    
taupc_histograms = taupc_histograms.astype(np.float32)
print(type(taupc_histograms[0,0,0]))

<type 'numpy.float32'>


In [9]:
# Loading file from harddrive
f = netCDF4.Dataset('/home/derektropf/Documents/NASA/code/ISCCP_HSeries_HGG_v01r00_198307_c20170705/ISCCP.HGG.v01r00.GLOBAL.1983.07.01.0000.GPC.10KM.CS00.EQ1.00.nc')
pctaudist = f['n_pctaudist']
sqlon_beg = f['sqlon_beg']
sqlon_end = f['sqlon_end']
eqlat = f['eqlat_index']
ntotal = f['n_total']

In [3]:
wsnums = np.zeros((360, 180, 12))
wscount = np.zeros((360, 180))
masks = np.zeros((6, 7, 41252), dtype=bool)
basepath = '/home/derektropf/Documents/NASA/code/ISCCP_HSeries_HGG_v01r00_198307_c20170705'
fullpath = '/home/derektropf/Documents/NASA/code/ISCCP_HSeries_HGG_v01r00_198307_c20170705/ISCCP.HGG.v01r00.GLOBAL.1983.07.01.0000.GPC.10KM.CS00.EQ1.00.nc'
for f in sorted(os.listdir(basepath)):
    #print(f)
    fullpath = basepath+'/'+f
    data = netCDF4.Dataset(fullpath)
    pctaudist = data['n_pctaudist']
    sqlonbeg = data['sqlon_beg'][:]
    sqlonend = data['sqlon_end'][:]
    eqlat = data['eqlat_index'][:]
    ntotal = data['n_total'][:]
    t1 = time.time()
    masks[:,:,:] = np.ma.getmaskarray(pctaudist[:,:,:])
    pctaudistnorm = analysis.fnorm(pctaudist[:,:,:], ntotal[:], masks[:,:,:])
    histograms = taupc_histograms
    wsnums, wscount = analysis.fcompare(pctaudistnorm[:,:,:], histograms[:,:,:]/100.0, masks[:,:,:], 
                                        wsnums[:,:], wscount[:], sqlonbeg[:], sqlonend[:], eqlat[:])
    t2 = time.time()
    print('run time for single 3 hour file: ', t2-t1)

('run time for single 3 hour file: ', 0.05396103858947754)
('run time for single 3 hour file: ', 0.0539247989654541)
('run time for single 3 hour file: ', 0.04650092124938965)
('run time for single 3 hour file: ', 0.04910397529602051)
('run time for single 3 hour file: ', 0.051586151123046875)
('run time for single 3 hour file: ', 0.04996299743652344)
('run time for single 3 hour file: ', 0.046623945236206055)
('run time for single 3 hour file: ', 0.044747114181518555)
('run time for single 3 hour file: ', 0.04513096809387207)
('run time for single 3 hour file: ', 0.04665708541870117)
('run time for single 3 hour file: ', 0.04338192939758301)
('run time for single 3 hour file: ', 0.044838905334472656)
('run time for single 3 hour file: ', 0.04380917549133301)
('run time for single 3 hour file: ', 0.04876089096069336)
('run time for single 3 hour file: ', 0.04761505126953125)
('run time for single 3 hour file: ', 0.045413970947265625)
('run time for single 3 hour file: ', 0.052057981491

In [8]:
fig, ax = plt.subplots()
sns.heatmap(np.flipud(wsnums[:,:,5].T))